In [1]:
%pip install tensorrt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.9/879.9 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 96.0 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.14.1.48.post1-py2.py3-none-any.whl size=16738 sha256=8ded7ab06be7c3d072ae31cb042fb6d80315361eda08c8903357055016b9e449
  Stored in directory: /root/.cache/pip/wheels/17/bf/f0/25cc47daa9fb0f17d17396b6c0201a4779b8b479974bd13cbd
  Created wheel for tensorrt_cu13: filename=tensorrt_cu13-10.14.1.48.post1-py2.py3-none-any.whl size=18162 sha256=2948f83af8d77c511572ba2c4028050d4cb5c0248feb0ebb741f7b7994658c75
  Stored in directory: /root/.cache/pip/wheels/d1/26/6f/efb7e2860ca12b2ddf1b73f69e5566e4e7aee35a235ac37cc3
  Created wheel for tensorrt_cu13_libs: filename=tensorrt_cu13_

In [3]:
# 1. Install dependencies for Linux/Colab
!pip install tensorrt onnx cuda-python pycuda

# 2. Create the project structure in Colab
import os
!mkdir -p models/onnx models/trt reports/profiler_plots data/sample_audio

# 3. Verify GPU
!nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.9 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 74.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 7.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2026.1-cp312-cp312-linux_x86_64.whl size=659448 sha256=2343889a894d5df49aa2a5939491b74c46b06d533fdda5c778f002d444e6ff5b
  Stored in directory: /root/.cache/pip/wheels/90/2a/71/75ec0cc316cc0ff494bfffa2935e02580129cb7f859a0cfd8f
Successfully built pycuda
Fri Jan 23 18:51:42 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|--------------------

In [ ]:
import tensorrt as trt
import os
import sys

def build_engine():
    # Fix: Use getcwd() if __file__ is missing (Notebook mode)
    if "__file__" in globals():
        script_dir = os.path.dirname(os.path.abspath(__file__))
        root = os.path.dirname(script_dir)
    else:
        # We assume you are running this from /content/ or the project root in Colab
        root = os.getcwd()

    onnx_file = os.path.join(root, "models", "onnx", "quantized_model.onnx")
    engine_file = os.path.join(root, "models", "trt", "model.engine")
    os.makedirs(os.path.dirname(engine_file), exist_ok=True)

    # Logger for debugging
    logger = trt.Logger(trt.Logger.VERBOSE)
    builder = trt.Builder(logger)
    
    # Flags for explicit batch (required for ONNX)
    network_flags = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
    network = builder.create_network(network_flags)
    parser = trt.OnnxParser(network, logger)
    config = builder.create_builder_config()

    print(f"Parsing ONNX: {onnx_file}")
    with open(onnx_file, 'rb') as model:
        if not parser.parse(model.read()):
            for error in range(parser.num_errors):
                print(parser.get_error(error))
            return

    # Optimization Profile (handling the dynamic audio length)
    profile = builder.create_optimization_profile()
    # Name must match your ONNX input name exactly
    profile.set_shape("input_values", (1, 16000), (1, 80000), (1, 480000))
    config.add_optimization_profile(profile)

    print("Building engine (this takes 2-5 minutes on T4)...")
    serialized_engine = builder.build_serialized_network(network, config)
    
    with open(engine_file, 'wb') as f:
        f.write(serialized_engine)
    print(f"✅ Success! Engine saved to {engine_file}")

if __name__ == "__main__":
    build_engine()

TypeError: pybind11::init(): factory function returned nullptr